In [1]:
import numpy as np 
import pandas as pd 
from ast import literal_eval
import os
import pickle
from collections import defaultdict
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import nltk
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical

In [2]:
# Load text from pickle file
with open('preprocessed_data.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
df['nama'] = df['nama'].str.lower()
df['nama'] = df['nama'].str.replace('[^\w\s]', '')

In [4]:
food_names = df['nama'].values
categories = df['kategori'].values

# Tokenize food names
tokenizer = Tokenizer()
tokenizer.fit_on_texts(food_names)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(food_names)
padded_sequences = pad_sequences(sequences)

# Encode categories using label encoding
label_encoder = LabelEncoder()
encoded_categories = label_encoder.fit_transform(categories)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_categories, test_size=0.2, random_state=42)

In [5]:
# Define the neural network architecture
embedding_dim = 50

model = keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=padded_sequences.shape[1]),
    Flatten(),
    Dropout(0.7),
    Dense(64, activation='relu'),
    Dense(len(set(encoded_categories)), activation='softmax')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
220/220 [==============================] - 1s 4ms/step - loss: 2.9027 - accuracy: 0.2067 - val_loss: 2.4832 - val_accuracy: 0.3222
Epoch 2/50
220/220 [==============================] - 1s 3ms/step - loss: 2.2404 - accuracy: 0.4018 - val_loss: 1.9581 - val_accuracy: 0.4920
Epoch 3/50
220/220 [==============================] - 1s 3ms/step - loss: 1.8414 - accuracy: 0.5109 - val_loss: 1.6932 - val_accuracy: 0.5540
Epoch 4/50
220/220 [==============================] - 1s 3ms/step - loss: 1.5846 - accuracy: 0.5712 - val_loss: 1.5448 - val_accuracy: 0.5710
Epoch 5/50
220/220 [==============================] - 1s 3ms/step - loss: 1.4324 - accuracy: 0.6099 - val_loss: 1.4528 - val_accuracy: 0.5898
Epoch 6/50
220/220 [==============================] - 1s 3ms/step - loss: 1.2945 - accuracy: 0.6364 - val_loss: 1.3789 - val_accuracy: 0.6159
Epoch 7/50
220/220 [==============================] - 1s 3ms/step - loss: 1.1889 - accuracy: 0.6676 - val_loss: 1.3184 - val_accuracy: 0.6335
Epoch 

In [6]:
model.save('recsys_version_1.h5')

In [9]:
#Get item latent factors
item_embedding_model = Model(inputs=model.input, outputs=model.layers[0].output)
item_latent_factors = item_embedding_model.predict(padded_sequences)
item_latent_factors = item_latent_factors.reshape((item_latent_factors.shape[0], -1))

# Calculate item-item similarity matrix
item_similarities = np.dot(item_latent_factors, item_latent_factors.T)

# Get recommendations for a specific food
query_food_name = 'bakso goreng'
query_food_index = np.where(food_names == query_food_name)[0][0]
query_item_similarities = item_similarities[query_food_index]
most_similar_indices = np.argsort(query_item_similarities)[-26:-1]  # Get top 20 most similar food indices
recommended_foods = np.unique(food_names[most_similar_indices])

# Print the recommendations
print(f"Recommendations for '{query_food_name}':")
for food in recommended_foods:
    print(food)

275/275 [==============================] - 0s 643us/step
Recommendations for 'bakso goreng':
bakso ayam
bakso goreng
dada ayam
kaki ayam
kornet ayam
naget ayam
paha ayam
pepes ayam
royco ayam
sayap ayam
siomay ayam
sozzis ayam
stik naget ayam
tepung bakso goreng
